In [84]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.feature_extraction import FeatureHasher

In [92]:
#hiperparametrosClusteringAvisos={ 'cantidadFeaturesTitulo' : 100, 'cantidadClusteresAvisos' : 10}

def get_features_titulo():
    return 100

def get_clusters_avisos():
    return 10

def get_avisos_detalle():
    avisos1 = pd.read_csv('data/fiuba_6_avisos_detalle.csv')
    avisos2 = pd.read_csv('data/FiubaDesde15Abril/fiuba_6_avisos_detalle.csv')
    avisos3 = pd.read_csv('data/FiubaHasta15Abril/fiuba_6_avisos_detalle.csv')
    avisos4 = pd.read_csv('data/fiuba_6_avisos_detalle_missing_nivel_laboral.csv')        
    avisos_detalle = pd.concat([avisos1,avisos2,avisos3,avisos4]).drop_duplicates(subset=['idaviso'], keep='last').reset_index(drop=True)
    columns_rename = {'idpostulante': 'id_postulante', 'idaviso': 'id_aviso'}
    avisos_detalle = avisos_detalle.rename(columns=columns_rename)
    to_nivel_laboral_nro = {'Senior / Semi-Senior' : 2, 'Junior':1, 'Otro':0,
       'Jefe / Supervisor / Responsable':3,
       'Gerencia / Alta Gerencia / Dirección':4}
    to_tipo_trabajo_nro={'Full-time':0, 'Part-time':1, 'Teletrabajo':2, 'Por Horas':3, 'Pasantia':4,
       'Temporario':5, 'Por Contrato':6, 'Fines de Semana':7, 'Primer empleo':8,
       'Voluntario':9}
    to_nombre_area_numero = pd.Series(avisos_detalle['nombre_area'].unique()).to_dict()
    to_nombre_area_numero  = {v: k for k, v in to_nombre_area_numero.items()}
    to_nombre_zona_numero = pd.Series(avisos_detalle['nombre_zona'].unique()).to_dict()
    to_nombre_zona_numero  = {v: k for k, v in to_nombre_zona_numero.items()}
    avisos_detalle['nivel_laboral_nro']= avisos_detalle['nivel_laboral'].map(to_nivel_laboral_nro)
    avisos_detalle['nivel_laboral_nro'].fillna(value =0, inplace = True)
    avisos_detalle['tipo_de_trabajo_nro'] = avisos_detalle['tipo_de_trabajo'].map(to_tipo_trabajo_nro)
    avisos_detalle['nombre_area_nro'] = avisos_detalle['nombre_area'].map(to_nombre_area_numero)
    avisos_detalle['zona'] = avisos_detalle['nombre_zona'].map(to_nombre_zona_numero)
    avisos_detalle['titulo_texto'] = avisos_detalle['titulo']
    h = FeatureHasher(n_features = get_features_titulo(), non_negative=True, input_type='string', dtype='int')
    x = h.transform(avisos_detalle['titulo'])
    avisos_detalle['titulo'] = list(x.toarray())
    titulos_como_lista = avisos_detalle.titulo.apply(pd.Series)
    avisos_detalle = pd.merge(avisos_detalle, titulos_como_lista, left_index = True, right_index = True)
    avisos_detalle = avisos_detalle.drop(['titulo'], axis=1)
    return avisos_detalle

def x_avisos():
    return ['tipo_de_trabajo_nro', 'nivel_laboral_nro', 'nombre_area_nro', 'zona'] + list(range(0,get_features_titulo()))

def columnas_para_leer():
    return ['descripcion', 'nombre_zona', 'tipo_de_trabajo', 'nivel_laboral', 'titulo_texto','nombre_area','denominacion_empresa', 'cluster']

In [77]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs

plt.figure(figsize=(12, 12))

n_samples = 1500
random_state = 170
X, y = make_blobs(n_samples=n_samples, random_state=random_state)

# Incorrect number of clusters
y_pred = KMeans(n_clusters=2, random_state=random_state).fit_predict(X)

In [3]:
X

array([[ -5.19811282e+00,   6.41869316e-01],
       [ -5.75229538e+00,   4.18627111e-01],
       [ -1.08448984e+01,  -7.55352273e+00],
       ..., 
       [  1.36105255e+00,  -9.07491863e-01],
       [ -3.54141108e-01,   7.12241630e-01],
       [  1.88577252e+00,   1.41185693e-03]])

In [93]:
avisos = get_avisos_detalle()
avisos.isnull().sum()

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\sklearn\feature_extraction\hashing.py:94: DeprecationWarning: the option non_negative=True has been deprecated in 0.19 and will be removed in version 0.21.
  " in version 0.21.", DeprecationWarning)


id_aviso                    0
idpais                      0
descripcion                 0
nombre_zona                 0
ciudad                  25128
mapacalle               23219
tipo_de_trabajo             0
nivel_laboral             335
nombre_area                 0
denominacion_empresa        7
nivel_laboral_nro           0
tipo_de_trabajo_nro         0
nombre_area_nro             0
zona                        0
titulo_texto                0
0                           0
1                           0
2                           0
3                           0
4                           0
5                           0
6                           0
7                           0
8                           0
9                           0
10                          0
11                          0
12                          0
13                          0
14                          0
                        ...  
70                          0
71                          0
72        

In [79]:
X = avisos[x_avisos()]

In [74]:
X.head()

,tipo_de_trabajo_nro,nivel_laboral_nro,nombre_area_nro,zona,0,1,2,3,4,5,...,10,11,12,13,14,15,16,17,18,19
0,0,2.0,0,0,0,4,1,5,1,4,...,3,0,8,0,0,1,1,2,0,1
1,0,3.0,1,1,0,5,1,2,6,3,...,7,0,11,1,1,0,0,3,0,6
2,0,1.0,2,0,0,4,1,0,2,1,...,1,1,2,0,0,0,1,2,0,2
3,0,1.0,3,0,0,8,1,6,1,3,...,1,0,7,0,1,0,2,2,0,1
4,0,2.0,1,0,0,4,1,2,3,4,...,8,0,10,0,1,0,1,2,0,9


In [80]:
y_pred = KMeans(n_clusters=get_clusters_avisos(), random_state=random_state).fit_predict(X)

In [82]:
avisos['cluster'] = y_pred

In [43]:
avisos.columns

Index([            'id_aviso',               'idpais',          'descripcion',
                'nombre_zona',               'ciudad',            'mapacalle',
            'tipo_de_trabajo',        'nivel_laboral',          'nombre_area',
       'denominacion_empresa',                 'zona',                      0,
                            1,                      2,                      3,
                            4,                      5,                      6,
                            7,                      8,                      9,
                           10,                     11,                     12,
                           13,                     14,                     15,
                           16,                     17,                     18,
                           19,              'cluster'],
      dtype='object')

In [83]:
avisos[avisos['cluster']==5][columnas_para_leer()]

,descripcion,nombre_zona,tipo_de_trabajo,nivel_laboral,titulo_texto,nombre_area,denominacion_empresa,cluster
107,<p>Nos encontramos en la búsqueda de Estudiant...,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Estudiantes Universitarios/Graduados de Cs. Ec...,Finanzas,J.P. Morgan,5
108,"<p style="""">Te estamos buscando para formar pa...",Gran Buenos Aires,Full-time,Senior / Semi-Senior,Cajero/a - Repositor/a PLAZO FIJO,Tesorería,Farmacity,5
110,<div><p>Empresa internacional referentes de la...,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Developer Sr (Ruby on Rails o Phyton),Programación,RANDSTAD,5
111,<p>Empresa internacional referente de la regió...,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Front end Web Developer Sr,Programación,RANDSTAD,5
112,<p>Nos encontramos en la búsqueda de un Desarr...,Gran Buenos Aires,Full-time,Junior,Desarrollador GeneXus Jr.,Programación,ML Soluciones Humanas,5
113,"<p>Moni Online, primera fintech de micropresta...",Capital Federal,Part-time,Junior,Gestor Cobranzas,Oficios y Profesiones,Moni Online,5
120,"<p style=""""><span style="""">¡En W3 te estamos b...",Gran Buenos Aires,Full-time,Senior / Semi-Senior,Desarrollador JAVA,Programación,W3,5
121,"<p>At Globant, we dream and build Digital Jour...",Gran Buenos Aires,Full-time,Senior / Semi-Senior,We are looking for a Android Developer to join...,Tecnologia / Sistemas,Globant,5
123,<p>Título: Odontólogos/as y ortodoncistas</p><...,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Odontologo/Ortodoncista Mar del Plata - Full time,Salud,AMOBA,5
124,<p>Importante empresa del rubro de transporte ...,Gran Buenos Aires,Full-time,Senior / Semi-Senior,A - Gomero,Oficios y Profesiones,Grupo Gestión,5
